<a href="https://colab.research.google.com/github/mulcahrj/DATA6545_Final/blob/main/TPOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TPOT

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path = 'https://raw.githubusercontent.com/mulcahrj/DATA6545_Final/main/selected_features_train.csv'
features_train = pd.read_csv(path)

path = 'https://raw.githubusercontent.com/mulcahrj/DATA6545_Final/main/selected_features_test.csv'
features_test = pd.read_csv(path)

In [ ]:
# import required packages for splitting data
from sklearn import model_selection
from sklearn import linear_model
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# balance the data
from imblearn.over_sampling import SMOTE

In [ ]:
frames = [features_test, features_train]

data = pd.concat(frames)

In [ ]:
data_shuffle=data.iloc[np.random.permutation(len(data))]
newdata=data_shuffle.reset_index(drop=True)
newdata.head()

,poutcome_success,emp.var.rate,euribor3m,cons.price.idx,cons.conf.idx,campaign,total_assets,rep_avg_rating,customer_length,qtr_bin_Q1,y
0,0,1.000000,0.968046,0.605052,0.408442,0.677663,0.665316,0.441981,0.519096,0,0
1,0,0.148320,0.067521,0.216596,0.235723,0.391017,0.990034,0.435268,0.514888,0,0
2,0,1.000000,0.968810,0.414483,0.692481,0.000000,0.472315,0.763135,0.804239,0,0
3,0,0.148320,0.069214,0.216596,0.235723,0.000000,0.542356,1.000000,0.999687,0,0
4,0,0.849418,0.931010,0.637496,0.680736,0.391017,0.694378,0.637057,0.695529,0,0


In [ ]:
data_class = newdata.values[:,10]

In [ ]:
# cross_validation train_test_split
from sklearn.model_selection import train_test_split

# split the data into training and testing (75% training, 25% testing, random state = fixed)
training_indices, testing_indices = train_test_split(newdata.index,stratify = data_class,train_size=0.75, test_size=0.25, random_state = 2019)

In [ ]:
##install TPOT
!pip install tpot

     |████████████████████████████████| 87 kB 3.2 MB/s 
     |████████████████████████████████| 192.9 MB 69 kB/s 
     |████████████████████████████████| 160 kB 80.4 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11956 sha256=50e2af21a96899d63f1a5d524ba8d2599615edd9697136a101dc57389baef17f
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
from tpot import TPOTClassifier
from sklearn.ensemble import RandomForestClassifier
# from tpot import TPOTRegressor # for regression tasks

tpot = TPOTClassifier(generations=5,max_time_mins=60, 
                      max_eval_time_mins=0.4, verbosity=2,  n_jobs=-1)

tpot.fit(newdata.drop('y',axis=1).loc[training_indices].values, # X_train
         newdata.loc[training_indices,'y'].values)

Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8995585638610948

Generation 2 - Current best internal CV score: 0.8995585638610948


#Evaluation

In [ ]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# balance the data
from imblearn.over_sampling import SMOTE

In [ ]:
# define features and target
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [ ]:
# Resampling and Balancing the data
sm = SMOTE(random_state = 2021) 
X_res, Y_res = sm.fit_resample(X, y) 

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier( bootstrap=True, criterion= 'entropy', max_features=0.8, min_samples_leaf=20, min_samples_split=5, n_estimators=100)
my_eval(X_res, Y_res, clf,10)

In [ ]:
#average ROC_AUC from 10-fold CV
clf = ExtraTreesClassifier( bootstrap=True, criterion= 'entropy', max_features=0.8, min_samples_leaf=20, min_samples_split=5, n_estimators=100)
my_eval(X_res, Y_res, clf, 10, 'roc_auc')